In [28]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt

In [55]:
def ewm_covariance_matrix_annualized(returns: pd.DataFrame, lookback_window=20) -> pd.DataFrame:
    """ Function to estimate ex-ante covariance matrix via exponentially weighted volatiilty.

    Args:
        returns (pd.DataFrame): _description_
        lookback_window (int, optional): _description_. Defaults to 20.

    Returns:
        pd.DataFrame: _description_
    """

    # Compute rolling EWMA volatility 
    vol = returns.ewm(span=lookback_window).std().iloc[-1] * np.sqrt(252)
    var = returns.ewm(span=lookback_window).var().iloc[-1] * 252

    # Get correlation matrix
    correlation = returns.tail(lookback_window).corr()

    # Compoenent-wise multiply volatility, then multiply with correlation coefficient to obtain covariance matrix
    covariance_matrix = correlation * np.outer(vol, vol)

    return covariance_matrix, var


# Assume we have a dataframe of daily returns for 4 assets
df_returns = pd.DataFrame(np.random.randn(252, 4), columns=['asset1', 'asset2', 'asset3', 'asset4'])

ewm_covariance_matrix(df_returns)


(            asset1      asset2      asset3      asset4
 asset1  313.646861  -18.563098  -35.368990 -103.537422
 asset2  -18.563098  232.220116   69.611234  -47.357780
 asset3  -35.368990   69.611234  113.471526   16.707104
 asset4 -103.537422  -47.357780   16.707104  335.331959,
 asset1    313.646861
 asset2    232.220116
 asset3    113.471526
 asset4    335.331959
 Name: 251, dtype: float64)

In [30]:
assets = ['SPY', 'QQQ', 'SPTM', 'MDY', 'IEFA', 'EEM', 'LQD', 'TLT', 'XLE', 'DBC', 'XLRE', 'GLD', 'IEF', 'SLV', 'DBA','JNK', 'HYG'] # , 'BIL']
# assets = ['BIL']
# Returns of opening prices
open_asset_returns = pd.DataFrame()
# Returns of close prices
close_asset_returns = pd.DataFrame()


for ticker in assets:
    prices = yf.download(ticker, start='1980-01-01', end=dt.date.today())
    
    # Get i'th asset's returns
    close_rets = prices['Adj Close'].pct_change().dropna()
    close_rets = close_rets.rename(ticker)
    open_rets = prices['Open'].pct_change().dropna()
    open_rets = open_rets.rename(ticker)

    close_asset_returns = pd.concat([close_asset_returns, close_rets], axis=1)
    open_asset_returns = pd.concat([open_asset_returns, open_rets], axis=1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [58]:
cov, var = ewm_covariance_matrix_annualized(close_asset_returns.iloc[:, :5], lookback_window=60)
cov

,SPY,QQQ,SPTM,MDY,IEFA
SPY,0.022111,0.026339,0.022298,0.024584,0.015610
QQQ,0.026339,0.037794,0.026216,0.025428,0.016045
SPTM,0.022298,0.026216,0.022632,0.025564,0.015914
MDY,0.024584,0.025428,0.025564,0.035869,0.019321
IEFA,0.015610,0.016045,0.015914,0.019321,0.020995


In [51]:
var

SPY     0.011550
QQQ     0.020448
SPTM    0.011660
MDY     0.021474
IEFA    0.011906
Name: 2023-04-21 00:00:00, dtype: float64

In [59]:
close_asset_returns.iloc[:, :5].cov()*252

,SPY,QQQ,SPTM,MDY,IEFA
SPY,0.035714,0.046273,0.034783,0.038743,0.025369
QQQ,0.046273,0.076883,0.040012,0.049045,0.027453
SPTM,0.034783,0.040012,0.040108,0.038520,0.024841
MDY,0.038743,0.049045,0.038520,0.047866,0.028662
IEFA,0.025369,0.027453,0.024841,0.028662,0.029055
